# Импорты

In [3]:
from utils import Disassembled_DDL, diff_DDLs, main_alerting_function, alter_table

In [2]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

import pandas as pd
import numpy as np
import subprocess

pd.DataFrame.iteritems = pd.DataFrame.items

import time

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import os
from datetime import datetime
import numpy as np

# Демонстрация основных модулей

In [3]:
spark = SparkSession.builder\
    .config("spark.driver.memory", "1g")\
    .config("spark.executor.memory", '2g')\
    .config("spark.executor.cores", '2')\
    .config("hive.exec.dynamic.partition", "true")\
    .config("hive.exec.dynamic.partition.mode", "nonstrict")\
    .enableHiveSupport()\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [6]:
spark.sql("""
DROP TABLE IF EXISTS new.test
""")

DataFrame[]

In [7]:
spark.sql("""
CREATE TABLE new.test (
  col2 DECIMAL(2,0),
  col3 DECIMAL(1,0))
USING orc
PARTITIONED BY (col2)
""")

DataFrame[]

In [8]:
ddl1 = spark.sql("SHOW CREATE TABLE new.test").toPandas().iloc[0,0]

In [9]:
ddl2 = spark.sql("SHOW CREATE TABLE new.stat").toPandas().iloc[0,0]

In [10]:
D_ddl1 = Disassembled_DDL(ddl1)

In [11]:
D_ddl2 = Disassembled_DDL(ddl2)

In [12]:
printed = diff_DDLs(D_ddl1, D_ddl2)

In [13]:
for key, value in printed.items():
    if value:
        print(value[1])

The following fields have been deleted from the table: col3, col2
The following fields have been added to the table
    num_partitions bigint
    flag bigint
    time bigint
    min_f bigint
    cpu string
    min_v double
    max_f bigint
    volume bigint
    mem string
    max_v double

The following partitions fields have been deleted from the table: col2

table_db_name
new.test
-------------------------------------
changed_on
-------------------------------------
new.stat



In [15]:
main_alerting_function(spark)

In [16]:
table_name = 'new.test'


new_DDL = """
CREATE TABLE new.test (
  col1 DECIMAL(1,0),
  col2 DECIMAL(1,0))
USING orc
PARTITIONED BY (col2)
"""

select_statement = """
SELECT NULL as col1,
       col2
FROM new.test__backup
"""

alter_table(spark, table_name, new_DDL, select_statement)

start testing DDLs
Success create table from file
Success drop table from file
Success create table new_DDL
Success drop table new_DDL
start proccess
start create backup table
start upload data to backup table


Succes cnt_old == cnt_backup
Finish upload data to backup table
Start drop and recreate main table
Success recreate
start upload data to main table


Success. Finish work


In [17]:
main_alerting_function(spark)

# Обучение

In [6]:
def generate_table(spark, name, num_lines):
    
    spark.sql(f"DROP TABLE IF EXISTS {name}")
    
    shape = (num_lines, 5)

    data = np.random.randint(0, 20, size=shape)
    df = pd.DataFrame(data, columns = [str(i) for i in range(shape[1])])

    df_spark = spark.createDataFrame(df)

    df_spark.coalesce(1).write.format('ORC').saveAsTable(name, partitionBy = [str(i) for i in range(shape[1])][-1])

    shape = (num_lines, 5)

    data = np.random.randint(0, 7, size=shape)
    df = pd.DataFrame(data, columns = [str(i) for i in range(shape[1])])

    df_spark = spark.createDataFrame(df)

    df_spark.coalesce(1).write.format('ORC').insertInto(name)

    shape = (num_lines, 5)

    data = np.random.randint(15, 20, size=shape)
    df = pd.DataFrame(data, columns = [str(i) for i in range(shape[1])])

    df_spark = spark.createDataFrame(df)

    df_spark.coalesce(1).write.format('ORC').insertInto(name)
    
    shape = (num_lines, 5)

    data = np.random.randint(2, 16, size=shape)
    df = pd.DataFrame(data, columns = [str(i) for i in range(shape[1])])

    df_spark = spark.createDataFrame(df)

    df_spark.coalesce(1).write.format('ORC').insertInto(name)
    
    shape = (num_lines, 5)

    data = np.random.randint(8, 12, size=shape)
    df = pd.DataFrame(data, columns = [str(i) for i in range(shape[1])])

    df_spark = spark.createDataFrame(df)

    df_spark.coalesce(1).write.format('ORC').insertInto(name)

In [22]:
def statistic(table_name):
    schema, name = table_name.split('.')
    path = f'/user/hive/warehouse/{schema}.db/{name}/'
    
    command = subprocess.check_output([f'hdfs dfs -du -s {path}'], shell=True)
    command = command.decode()
    
    mem_without_replic = int(command.split('  ')[0])
    
    command = subprocess.check_output([f'hdfs dfs -find {path} -name "*.orc"'], shell=True)
    command = command.decode()
    
    all_files = command.split('\n')[:-1]
    
    all_dirs = set(map(lambda x: '/'.join(x.split('/')[:-1]), all_files))
    
    cnt_files_in_directory = {}
    
    for el in all_dirs:
        cnt_files_in_directory[el] = 0
    
    for el in all_files:
        key = '/'.join(el.split('/')[:-1])
        cnt_files_in_directory[key] += 1
    
    min_files = min(cnt_files_in_directory.values())
    max_files = max(cnt_files_in_directory.values())
    
    min_value = min_files * (int(mem_without_replic)/len(all_files))
    max_value = max_files * (int(mem_without_replic)/len(all_files))
                       
    all_parts = len(all_dirs)
    
    return (mem_without_replic, all_parts, min_files, max_files, min_value, max_value)


In [8]:
def test_table(name, num_lines, core, memory):
    t1 = time.time()
    
    spark = SparkSession.builder\
        .config("spark.driver.memory", "1g")\
        .config("spark.executor.memory", str(memory))\
        .config("spark.executor.cores", str(core))\
        .config("hive.exec.dynamic.partition", "true")\
        .config("hive.exec.dynamic.partition.mode", "nonstrict")\
        .enableHiveSupport()\
        .getOrCreate()
    
    generate_table(spark, name, num_lines)
    
    st = list(statistic(name))
    
    spark.sql(f"DROP TABLE IF EXISTS {name}_tmp")
    
    spark.sql(f"CREATE TABLE {name}_tmp LIKE {name}")
    
    try:
        spark.sql(f"SELECT * FROM {name}").write.format('ORC').insertInto(f'{name}_tmp')
    except:
        spark.sql(f"DROP TABLE {name}_tmp")
        spark.sql(f"DROP TABLE {name}")
        spark.stop()
        
        st.append(core)
        st.append(memory)
        st.append(0)
        t2 = time.time()
        st.append(round(t2 - t1))
        return st
    
    spark.sql(f"DROP TABLE {name}_tmp")
    spark.sql(f"DROP TABLE {name}")
    spark.stop()

    st.append(core)
    st.append(memory)
    st.append(1)
    t2 = time.time()
    st.append(round(t2 - t1))
    return st

test_table('new.test2', 10_000, '2', '2g')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[183406, 20, 2, 3, 7336.24, 11004.36, '2', '2g', 1, 33]

In [ ]:
a = []
for i in range(1, 6):
    for j in range(1, 8):
        for l in range(10_000, 500_000, 30_000):
            a.append(test_table('new.test2', l, f'i', f'{j}g'))

In [43]:
df = pd.DataFrame(a, columns = ['volume', 'num_partitions', 'min_f', 'max_f', 'min_v', 'max_v', 'cpu', 'mem', 'flag', 'time'])

In [4]:
spark = SparkSession.builder\
        .config("spark.driver.memory", "1g")\
        .config("spark.executor.memory", '4g')\
        .config("spark.executor.cores", 4)\
        .config("hive.exec.dynamic.partition", "true")\
        .config("hive.exec.dynamic.partition.mode", "nonstrict")\
        .enableHiveSupport()\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [44]:
df_spark = spark.createDataFrame(df)

df_spark.coalesce(1).write.format('ORC').saveAsTable('new.stat')

In [49]:
spark.sql("""
SELECT * 
FROM new.stat
LIMIT 3
""").toPandas()

,volume,num_partitions,min_f,max_f,min_v,max_v,cpu,mem,flag,time
0,183709,10,1,3,7353.64,11030.46,2,2g,1,16
1,1525973,15,2,7,61038.92,91558.38,3,4g,1,35
2,2998802,20,3,3,119952.08,179928.10,4,7g,1,60


In [119]:
df = spark.sql("""
SELECT volume,
       num_partitions,
       min_f,
       max_f,
       min_v,
       max_v,
       cpu,
       mem
FROM(
    SELECT *,
        ROW_NUMBER() over(PARTITION BY volume order by time) as d_id
    FROM new.stat
    )
WHERE d_id <= 3
""").toPandas()

In [120]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [121]:
df['mem'] = df['mem'].map(lambda x: int(x.replace('g','')))

In [122]:
cols = df.select_dtypes(exclude=['float']).columns
df[cols] = df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')

In [123]:
X = df[['num_partitions', 'volume', 'min_f', 'max_f', 'min_v', 'max_v']]

In [124]:
y = df['cpu']

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [126]:
reg_cpu = LinearRegression().fit(X_train, y_train)

In [129]:
reg_cpu.intercept_

1.8443069529099603

In [130]:
reg_cpu.coef_

array([ 5.34240582e-04,  7.67071198e-05,  1.20037864e-04,  1.02558832e-02,
        1.47215567e-03, -2.24814088e-03])

In [131]:
y_mem = df['mem']

In [132]:
X_trаin_mem, X_test, y_trаin_mem, y_test = train_test_split(X, y_mem, test_size=0.1, random_state=42)

In [133]:
reg_mem = LinearRegression().fit(X_train_mem, y_train_mem)

In [135]:
reg_mem.intercept_

1.946123654799294

In [136]:
reg_mem.coef_

array([-0.00468423, -0.00030779, -0.00105881, -0.09042189, -0.01664148,
        0.01625429])

In [67]:
def table_parametrs(table_name):
    stat = np.array(statistic(table_name))
    
    b1_mem = np.array([-0.00468423, -0.00030779, -0.00105881, -0.09042189, -0.01664148, 0.01625429])
    b0_mem = 1.946123654799294
    
    b1_cpu = np.array([5.34240582e-04,  7.67071198e-05,  1.20037864e-04,  1.02558832e-02, 1.47215567e-03, -2.24814088e-03])
    b0_cpu = 1.8443069529099603
    
    pred_mem = sum(stat*b1_mem) + b0_mem
    pred_cpu = sum(stat*b1_cpu) + b0_cpu
    
    if pred_mem > 7:
        pred_mem = 7
    if pred_mem < 1:
        pred_mem = 1
        
    if pred_cpu > 5:
        pred_cpu = 5
    if pred_cpu < 1:
        pred_cpu = 1
    
    return (int(pred_mem), int(pred_cpu))
table_parametrs('new.stat')

(1, 1)